In [1]:
# get a generic rpi 64 lease

To report a broken pi get the uuid of the pi and send to Mike or #edge-iot Slack
Rule of thumb: When getting a lease specify pi?
What devices are available?
* The host calendar



In [4]:
import chi
from chi import container
from chi import lease
# Before we go any further, we need to select which Chameleon site we will be using.
chi.use_site("CHI@Edge")

ModuleNotFoundError: No module named 'chi'

In [29]:
chi.set("project_name", "CHI-220971")
print(chi)

<module 'chi' from '/opt/conda/lib/python3.9/site-packages/chi/__init__.py'>


In [42]:
machine_name = "raspberrypi4-64"
start, end = lease.lease_duration(days=2)
reservations = []
lease.add_device_reservation(reservations, count=1, device_name="iot-rpi4-03", machine_name=machine_name)
container_lease = lease.create_lease(f"{machine_name}-{start}", reservations)
lease_id = container_lease["id"]

print("Waiting for lease to start ...")
lease.wait_for_active(lease_id)
print("Done!")

Waiting for lease to start ...
Done!


In [43]:
try:
    container.destroy_container("jupyter-server-roa")
except:
    pass

jupyter_server = None
print("Requesting Jupyter server container ...")
jupyter_server = container.create_container("jupyter-server-roa", 
    image="jupyter/minimal-notebook:aarch64-latest",
    #entrypoint=["tini", "-g", "--"],
    #command=["start-singleuser.sh"],
    #command=["tini", "-g", "--", "jupyter", "notebook", "--ip", "0.0.0.0", "--port", "8888", "--no-browser", "--allow-root"],
    #command=["ls"],
    command=["jupyter", "notebook", "--ip", "0.0.0.0", "--port", "8888", "--no-browser", "--allow-root"],                                     
    exposed_ports=[8888],
    reservation_id=lease.get_device_reservation(lease_id),
    platform_version=2)
print("Done!")


Requesting Jupyter server container ...
Done!


In [38]:
print("Requesting container ...")

my_container = container.create_container(
    "webserver-container", 
    image="arm64v8/python:latest",
    command=["python", "-m", "http.server", "8888"],
    workdir="/var/www/html",
    exposed_ports=[8888],
    reservation_id=lease.get_device_reservation(lease_id),
    platform_version=2,
)

print("Done!")

Requesting container ...
Done!


In [40]:
public_ip = container.associate_floating_ip(my_container.uuid)

print(f"http://{public_ip}:8888")

Conflict: Cannot associate floating IP 129.114.34.217 (3acdd521-47a1-4177-b450-f5436c66f24e) with port 1d314e75-09b7-4e15-952b-687989924239 using fixed IP 192.168.64.52, as that fixed IP already has a floating IP on external network 17446dec-0c72-4d28-abf5-99f43e152221.
Neutron server returns request_ids: ['req-92ecf4a1-5d2d-49e2-a29f-ef05ec10ff50']

In [41]:
print(my_container.uuid)


09aff40b-02b4-44ea-b23b-3cfda53176b6


In [ ]:
jupyter_server_private_ip = list(jupyter_server.addresses.values())[0][0]["addr"]
print(f"Private IP: {jupyter_server_private_ip}")

In [44]:
public_ip = container.associate_floating_ip(jupyter_server.uuid)

print(f"http://{public_ip}:8888")

http://129.114.34.217:8888


In [12]:
!apt-get install curl

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [11]:
!curl 192.168.64.55:8888/lab?token=0cf3806ab47cc3a79aae9c3557fcce8880807f188479a672


curl: (28) Failed to connect to 192.168.64.55 port 8888: Connection timed out


In [ ]:
!curl http://129.114.34.214:8888/lab?token=0cf3806ab47cc3a79aae9c3557fcce8880807f188479a672

In [16]:
#lease.delete_lease(lease_id)

Deleted lease with id 151825fe-2bd3-4eef-abd5-7916fc1c989d
